In [1]:
import sys 
sys.path.append('/home/sean/Documents/Work/Level 4/Level-4-Masters-Project/')
import tensorflow as tf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras import models 
import os
from keras.regularizers import l2
import matplotlib
from multiprocessing import Pool
import Hyperparam_Testing.Testing_Notebooks.Common_Functions as cf 
import Hyperparam_Testing.Testing_Notebooks.chirallag as cL

matplotlib.rcParams.update({'legend.fontsize': 16})
matplotlib.rcParams.update({'font.size': 16})


In [2]:
def gen_and_load_noisy(n_pred,n_val,sigma):
    number_predictions= n_pred
    N = 3
    F0 = 1
    gens = cL.gen_gellman(3)
    pi=np.random.rand(number_predictions,N*N-1)**0.25
    dpi=np.random.rand(number_predictions,N*N-1)**0.25
    orig_V = abs(cL.get_V(pi,dpi,gens,F0).real)
    #Make the xvals noisey
    pi = cf.apply_noise(pi,sigma)
    dpi = cf.apply_noise(dpi,sigma)
    output = np.hstack((pi,dpi,np.expand_dims(orig_V,axis=1)))
    
    return [(output[:-n_val,:-1],output[:-n_val,-1]),(output[-n_val:,:-1],output[-n_val:,-1])]

In [3]:
sigma = 0.03
data = gen_and_load_noisy(10**7,10**5,sigma)
train_x ,train_y = data[0]
val_x,val_y = data[1]

In [5]:
opt = keras.optimizers.Adam(beta_1= 0.9, beta_2= 0.98)


In [6]:

model = models.Sequential()

model.add(layers.Input(train_x.shape[1]))
model.add(layers.Dense(300,activation= 'relu',))
model.add(layers.Dense(300,activation= 'relu',))
model.add(layers.Dense(300,activation= 'relu',))
model.add(layers.Dense(300,activation= 'relu',))
model.add(layers.Dense(300,activation= 'relu',))
model.add(layers.Dense(1))
model.compile(optimizer = opt,loss = 'mape', metrics = [['mean_absolute_error'],['mean_absolute_percentage_error']])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 300)               5100      
_________________________________________________________________
dense_1 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_3 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_4 (Dense)              (None, 300)               90300     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 301       
Total params: 366,601
Trainable params: 366,601
Non-trainable params: 0
__________________________________________________

2022-02-16 22:47:47.139746: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-16 22:47:47.144827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-16 22:47:47.145102: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-16 22:47:47.145510: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [7]:
epoch_num = 100


(9900000, 16)

In [8]:

model.fit(train_x,train_y,validation_data=(val_x,val_y),batch_size= 32 , epochs = epoch_num)

2022-02-16 22:47:48.067457: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
309375/309375 [==============================] - 306s 988us/step - loss: 18.4838 - mean_absolute_error: 0.0212 - mean_absolute_percentage_error: 18.4838 - val_loss: 17.0576 - val_mean_absolute_error: 0.0195 - val_mean_absolute_percentage_error: 17.0576
Epoch 2/100
309375/309375 [==============================] - 309s 1000us/step - loss: 17.4311 - mean_absolute_error: 0.0199 - mean_absolute_percentage_error: 17.4311 - val_loss: 16.9003 - val_mean_absolute_error: 0.0194 - val_mean_absolute_percentage_error: 16.9003
Epoch 3/100
309375/309375 [==============================] - 309s 998us/step - loss: 17.2256 - mean_absolute_error: 0.0196 - mean_absolute_percentage_error: 17.2256 - val_loss: 17.0130 - val_mean_absolute_error: 0.0196 - val_mean_absolute_percentage_error: 17.0130
Epoch 4/100
309375/309375 [==============================] - 310s 1ms/step - loss: 17.1147 - mean_absolute_error: 0.0195 - mean_absolute_percentage_error: 17.1147 - val_loss: 16.6446 - val_mean_absolute_e

In [ ]:
#history = model.history
model.save('./5_10{}datapoints_noise_{}.h5'.format((np.log10(train_y.shape[0]+val_y.shape[0])),sigma))
#model = tf.keras.models.load_model('./batchsize16.h5')